# Sorting Contours

In [1]:
import cv2
import numpy as np

In [5]:
# Loading image
image = cv2.imread("../images/bunchofshapes.jpg")
cv2.imshow("Original",image)
cv2.waitKey()

# Making copy of original image
original_image = image

# Creating a black image with the same dimensions as of loaded image
black_image = np.zeros((image.shape[0],image.shape[1],3))
# cv2.imshow("Black Image",black_image)
# cv2.waitKey()

# Converting original image to grayscale
gray_image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

# Finding Canny Edges
edged = cv2.Canny(gray_image,50,100)

# Finding Contours
contours,hierarchy = cv2.findContours(edged,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)

# Printing the number of contours found
print("Number of contours found:",len(contours))

# Drawing all contours on black image
cv2.drawContours(black_image,contours,-1,(0,255,0),3)
cv2.imshow("Contours over black image",black_image)
cv2.waitKey()

# Drawing all contours on original image
cv2.drawContours(original_image,contours,-1,(0,255,0),3)
cv2.imshow("Contours over original image",original_image)
cv2.waitKey()
cv2.destroyAllWindows()

Number of contours found: 4


## Sorting Contours by Area

In [10]:
# Function that returns an array of areas of all contours
def get_contour_area(contours):
    all_areas = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        all_areas.append(area)
    return all_areas

# Loading image
image = cv2.imread("../images/bunchofshapes.jpg")

# Printing the areas of the contours before sorting
print("Areas of the contours before sorting:",get_contour_area(contours))

# Sorting the contours
sorted_contours = sorted(contours,key=cv2.contourArea,reverse=True)

# Printing the areas of the contours after sorting
print("Areas of the contours after sorting:",get_contour_area(sorted_contours))

# Iterating over the contours and drawing one at a time
for cnt in sorted_contours:
    cv2.drawContours(image,[cnt],-1,(255,0,0),3)
    cv2.imshow("Contours",image)
    cv2.waitKey()
cv2.destroyAllWindows()

Areas of the contours before sorting: [20587.5, 22900.5, 66581.5, 90222.0]
Areas of the contours after sorting: [90222.0, 66581.5, 22900.5, 20587.5]


## Sorting Contours by Position

In [24]:
# Loading image and making a copy
image = cv2.imread("../images/bunchofshapes.jpg")
original_image = image.copy()

# This function returns the x_cord for the contour centroid
def x_cord_contour(contour):
    if cv2.contourArea(contour) > 10:
        M = cv2.moments(contour)
        return (int(M['m10']/M['m00']))
    else:
        pass
    
# This function places a red circle at the center of contours
def label_contour_center(image,contour):
    M = cv2.moments(contour)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    image = cv2.circle(image,(cx,cy),10,(0,0,255),-1)
    return image

# Drawing red circles at the center of each contours
for cnt in contours:
    label_contour_center(image,cnt)
    
# cv2.imshow("Contour Centers",image)
# cv2.waitKey()

# Using our x_cord_contour function to sort the contours left to right
sorted_contours = sorted(contours,key=x_cord_contour,reverse=False)

# labeling contours left to right
for (i,cnt) in enumerate(sorted_contours):
    cv2.drawContours(original_image,[cnt],-1,(0,0,255),3)
    M = cv2.moments(cnt)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    cv2.putText(original_image,str(i+1),(cx,cy),cv2.FONT_HERSHEY_DUPLEX,1,(0,255,0),3)
    cv2.imshow("Contour with Labels",original_image)
    cv2.waitKey()
    
    # Cropping each contours and saving the images
    (x,y,w,h) = cv2.boundingRect(cnt)
    cropped_contour = original_image[y:y+h,x:x+w]
    image_name = "ouput_shape_"+str(i+1)+".jpg"
    cv2.imwrite(image_name,cropped_contour)
cv2.destroyAllWindows()